In [45]:
import pandas as pd
import numpy as np
from random import shuffle 


**Задача 1. Классическая задача парадокса дней рождений: определить вероятность того, что в группе, состоящей из 23 человек, у двух людей будет совпадение дней рождений(число и месяц).**


Решение:
Вероятность- если рандомно собирать группы из 23 человек и проверять совпадают ли у кого-то дни рождения -  в скольки процентах случаях будет это совпадение?

In [20]:
birthday = pd.Series(range(365))

Создадим генератор групп, в которых 23 человека, у которых дни рождения могут совпадать:

In [30]:
groups = [birthday.sample(23,replace=True).duplicated().max() for _ in range(10000)]

Посчитаем долю вероятности совпадения дней рождений.

In [31]:
np.mean(groups)

0.5154

**Ответ: Вероятность, что в группе, состоящей из 23 человек,  у двух людей будет совпадение дней рождений - 50%. Текст, выделенный полужирным шрифтом** 

**Задача 2. Экзамен проходит по следующей схеме: если некоторый билет уже был вытянут, то после ответа экзаменатор откладывает его в сторону. Студент выучил 20 билетов из 30. Когда ему выгоднее идти- первым или вторым, чтобы вероятность вынуть выученный билет была больше.**

In [46]:
n=10000
tickets = list(range (1, 31)) # билеты
student = list(range (1,21)) # билеты, которые выучил студент
result = []
for _ in range(n):
  shuffle(tickets) # перемешиваем билеты
  result.append(tickets[0]in student)
np.mean(result)

0.6727

Ответ:Вероятность вытянуть выученный билет  - 67%, без разницы студент идет первым или вторым. Текст, выделенный полужирным шрифтом

**Задача 3. Ночью произошло ДТП с участием такси. В городе работают две компании такси — «Синие» и «Зелёные». «Зелёным» принадлежит 85% такси, «Синим» 15%. Свидетель аварии утверждает, что такси принадлежало «Синим». Следственный эксперимент показал, что ночью свидетель верно определяет цвет такси в 80%. Какова вероятность того, что такси действительно принадлежало «Синим»?**

In [48]:
# обозначим "Синих" как единицу:
np.random.binomial(1, 0.15)

def witness(taxi): 
  if np.random.binomial(1, 0.8): # свидетель верно определяет цвет, только в 80%
    return taxi 
  return abs(taxi - 1) # если определяет неверно, то возвращается обратное значение

In [55]:
n = 10000
result = []
for _ in range(n):
  taxi = np.random.binomial(1, 0.15)
  witness_answer  = witness(taxi)
  result.append((taxi,witness_answer))

In [57]:
df = pd.DataFrame(result, columns=['taxi', 'witness_answer'])
df.head()

,taxi,witness_answer
0,0,1
1,1,1
2,0,0
3,1,1
4,1,1


In [60]:
df.groupby('witness_answer')['taxi'].mean()

witness_answer
0    0.044219
1    0.420145
Name: taxi, dtype: float64

**Ответ: Вероятность того, что такси принадлежало "Синим" - 42%.**

**Задача 4. Русская рулетка. Есть револьвер с двумя вставленными подряд патронами.
Первый крутит барабан и остается жив. Твоя очередь. Тебе предлагается покрутить барабан перед выстрелом. Согласишься или сразу выстрелишь и почему?**

In [153]:
# функция выстрела
def one_turn(chamber):
  new_chamber = [1,1,0,0,0,0]
  n=len(chamber)
  for i in range(n):
      if i< n-1:
          new_chamber[i+1] = chamber[i]
      else:
          new_chamber[0] = chamber[i]
  return new_chamber 

In [154]:
# функция вращения барабана
def spin_chamber(chamber):
  n = np.random.randint(1,7)
  for i in range(n):
    chamber = np.roll(chamber, n)
  return chamber

In [155]:
n = 10000
result = []
for _ in range(n):
    # заряжаем револьвер
    chamber = [1,1,0,0,0,0]
    # первый игрок крутит барабан
    chamber = spin_chamber(chamber)
    # первый игрок стреляет
    chamber = one_turn(chamber)
    p1 = chamber[0]
    chamber[0] = 0
    # второй игрок вращает барабан
    chamber = spin_chamber(chamber)
    # второй игрок стреляет
    chamber = one_turn(chamber)
    p2 = chamber[0]
    result.append((p1,p2))

In [156]:
df = pd.DataFrame(result, columns = ['p1','p2'])

In [157]:
df.groupby('p1')['p2'].agg(['count', 'mean'])

,count,mean
p1,,
0,6733,0.334918
1,3267,0.163453


Вероятность того, что револьвер выстрелит, если я покручу барабан составляет  33%.

In [158]:
n = 10000
result = []
for _ in range(n):
    # заряжаем револьвер
    chamber = [1,1,0,0,0,0]
    # первый игрок крутит барабан
    chamber = spin_chamber(chamber)
    # первый игрок стреляет
    chamber = one_turn(chamber)
    p1 = chamber[0]
    chamber[0] = 0
    # второй игрок не вращает барабан
    #chamber = spin_chamber(chamber)
    # второй игрок стреляет
    chamber = one_turn(chamber)
    p2 = chamber[0]
    result.append((p1,p2))

In [160]:
df = pd.DataFrame(result, columns = ['p1','p2'])
df.groupby('p1')['p2'].agg(['count', 'mean'])


,count,mean
p1,,
0,6601,0.248599
1,3399,1.000000


Вероятность того, что револьвер выстрелит, если я не покручу барабан составляет 25%.

**Ответ: не нужно крутить барабан, тогда вероятность выжить больше.**